In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [2]:
geo = SplineGeometry()
m = 0.6
geo.AddCircle(c=(0, 0), r=1, bc="cyl", maxh=m) # 0.04
mesh = Mesh(geo.GenerateMesh(maxh=m))
mesh2 = Mesh(geo.GenerateMesh(maxh=m))
mesh.Curve(3);
mesh2.Curve(3)
scene1 = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [3]:
# vector space for shape gradient
VEC = H1(mesh, order=1, dim=2)
# Test and trial functions
PHI, X = VEC.TnT()
gfX = GridFunction(VEC)

In [4]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

dJOmega  = LinearForm(VEC)

# volume stuff
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

# bc prep
bc_tx = CoefficientFunction(x)
bc_ty = CoefficientFunction(y)
bc_0x = 1/surf_0*Integrate(bc_tx,mesh)
bc_0y = 1/surf_0*Integrate(bc_ty,mesh)

# bc constraints
beta0 = 1e-3
beta = Parameter(beta0)
bc_x = Parameter(1)
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))

dJOmega += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)

In [5]:
dJOmega.Assemble()
print(surf_0)
print(dJOmega.vec[0:5])

3.1417066211235207
       0
      0

       0
      0

       0
      0

       0
      0

       0
      0




In [6]:
test = CoefficientFunction((2*x+1,0.5*x))
gfX.Set(test)
mesh.SetDeformation(gfX)
Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

BaseWebGuiScene

In [7]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
dJOmega.Assemble()
print(bc_x.Get())
print(abs(bc_0x-bc_x.Get()))
print(dJOmega.vec)

3.008598371293703
3.008598371293703
  0.00229899
 -0.0137204

  0.00452441
 -2.28952e-18

  -0.00229899
 0.0137204

  -0.00452441
 -1.99058e-18

  0.00435703
 -0.0121723

  0.00523175
 -0.00699633

  0.00289872
 0.00699633

  0.000303506
 0.0121723

  -0.00435703
 0.0121723

  -0.00523175
 0.00699633

  -0.00289872
 -0.00699633

  -0.000303506
 -0.0121723

  -4.33681e-19
 3.46945e-18

  4.33681e-19
 -8.67362e-19

  8.67362e-19
 -5.42101e-20

  3.52366e-19
 -1.73472e-18

  -8.67362e-19
 5.42101e-20

  5.14996e-19
 -1.73472e-18

  8.67362e-19
 8.67362e-19


